In [1]:
import pandas as pd
import pandas as pd
import gensim
from gensim import corpora, models
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
train=pd.read_csv("./Data Set/train.csv")
test=pd.read_csv("./Data Set/test.csv")
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [3]:
train['Text']=train['TITLE'] + train['ABSTRACT']
train['Text']

0        Reconstructing Subject-Specific Effect Maps  P...
1        Rotation Invariance Neural Network  Rotation i...
2        Spherical polyharmonics and Poisson kernels fo...
3        A finite element approximation for the stochas...
4        Comparative study of Discrete Wavelet Transfor...
                               ...                        
20967    Contemporary machine learning: a guide for pra...
20968    Uniform diamond coatings on WC-Co hard alloy c...
20969    Analysing Soccer Games with Clustering and Con...
20970    On the Efficient Simulation of the Left-Tail o...
20971    Why optional stopping is a problem for Bayesia...
Name: Text, Length: 20972, dtype: object

In [4]:
article = train['Text'].tolist()
labels = train[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)
data_words= list(sent_to_words(article))

In [6]:
data_words[:2]

[['reconstructing',
  'subject',
  'specific',
  'effect',
  'maps',
  'predictive',
  'models',
  'allow',
  'subject',
  'specific',
  'inference',
  'when',
  'analyzing',
  'disease',
  'related',
  'alterations',
  'in',
  'neuroimaging',
  'data',
  'given',
  'subject',
  'data',
  'inference',
  'can',
  'be',
  'made',
  'at',
  'two',
  'levels',
  'global',
  'identifiying',
  'condition',
  'presence',
  'for',
  'the',
  'subject',
  'and',
  'local',
  'detecting',
  'condition',
  'effect',
  'on',
  'each',
  'individual',
  'measurement',
  'extracted',
  'from',
  'the',
  'subject',
  'data',
  'while',
  'global',
  'inference',
  'is',
  'widely',
  'used',
  'local',
  'inference',
  'which',
  'can',
  'be',
  'used',
  'to',
  'form',
  'subject',
  'specific',
  'effect',
  'maps',
  'is',
  'rarely',
  'used',
  'because',
  'existing',
  'models',
  'often',
  'yield',
  'noisy',
  'detections',
  'composed',
  'of',
  'dispersed',
  'isolated',
  'islands',


In [7]:
def preprocess(text):
    custom_filters = [
        lambda x: x.lower(),          # Convert to lowercase
        lambda x: x.strip(),          # Remove leading/trailing whitespaces
        remove_stopwords,             # Remove stop words
    ]

    processed_texts = []
    for item in text:
        processed_text = item
        for filter_func in custom_filters:
            processed_text = filter_func(processed_text)
        processed_texts.append(processed_text)  # Append the processed tokens as a list
    
    # Generate bigrams and trigrams
    bigram = Phrases(processed_texts, min_count=5, threshold=100)
    trigram = Phrases(bigram[processed_texts], min_count=5, threshold=100)

    # Apply bigrams and trigrams to texts
    processed_texts = trigram[bigram[processed_texts]]

    return processed_texts

In [8]:
documents = [preprocess(text) for text in data_words]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([' '.join(doc) for doc in documents])


In [11]:
from gensim.models import LdaModel

In [12]:
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
id2word = {index: word for index, word in enumerate(vectorizer.get_feature_names_out())}
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=5)


In [13]:
topics = lda_model.show_topics(num_topics=5, num_words=5)
for topic in topics:
    print(topic)

(0, '0.006*"problem" + 0.005*"paper" + 0.005*"space" + 0.005*"prove" + 0.005*"results"')
(1, '0.013*"data" + 0.010*"model" + 0.009*"learning" + 0.008*"based" + 0.007*"network"')
(2, '0.009*"image" + 0.007*"images" + 0.006*"based" + 0.006*"neural" + 0.005*"language"')
(3, '0.007*"model" + 0.006*"phase" + 0.005*"magnetic" + 0.005*"spin" + 0.004*"field"')
(4, '0.005*"data" + 0.005*"mass" + 0.004*"high" + 0.004*"based" + 0.004*"large"')


In [14]:
labels = labels.idxmax(axis=1)

In [15]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([' '.join(doc) for doc in documents])


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [19]:


# Support Vector Machine Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)
svm_y_pred = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
print("Support Vector Machine Classifier Accuracy:", svm_accuracy)



Support Vector Machine Classifier Accuracy: 0.797139451728248


In [21]:
predicted_topics = lda_model.get_document_topics(corpus)
topic_names = {0: "Computer Science", 1: "Physics", 2: "Mathematics", 3: "Statistics", 4: "Quantitative Biology", 5: "Quantitative Finance"}
for i, doc_topics in enumerate(predicted_topics):
    topic = max(doc_topics, key=lambda x: x[1])[0]
    topic_name = topic_names[topic]
    print(f"Article {i+1}: Predicted Topic: {topic_name}")

Article 1: Predicted Topic: Physics
Article 2: Predicted Topic: Mathematics
Article 3: Predicted Topic: Computer Science
Article 4: Predicted Topic: Computer Science
Article 5: Predicted Topic: Physics
Article 6: Predicted Topic: Computer Science
Article 7: Predicted Topic: Quantitative Biology
Article 8: Predicted Topic: Statistics
Article 9: Predicted Topic: Statistics
Article 10: Predicted Topic: Statistics
Article 11: Predicted Topic: Statistics
Article 12: Predicted Topic: Statistics
Article 13: Predicted Topic: Statistics
Article 14: Predicted Topic: Computer Science
Article 15: Predicted Topic: Statistics
Article 16: Predicted Topic: Computer Science
Article 17: Predicted Topic: Quantitative Biology
Article 18: Predicted Topic: Computer Science
Article 19: Predicted Topic: Physics
Article 20: Predicted Topic: Computer Science
Article 21: Predicted Topic: Statistics
Article 22: Predicted Topic: Statistics
Article 23: Predicted Topic: Physics
Article 24: Predicted Topic: Computer 

In [44]:
new_doc = "This is a new document about Computer Science"
processed_new_doc = preprocess(new_doc)
new_doc_vector = vectorizer.transform([' '.join(processed_new_doc)]).todense()


In [51]:
new_doc_corpus = gensim.matutils.Dense2Corpus(new_doc_vector, documents_columns=False)
if topic_names:
    max_topic = max(topic_names, key=lambda x: x[1])[0]
    predicted_topic_name = topic_names.get(max_topic, "Unknown")
    print("LDA Predicted Topic:", predicted_topic_name)
else:
    print("No LDA topic prediction for the given document")

TypeError: 'int' object is not subscriptable